In [ ]:
#Seas DBG PIV Code - Frame Average
#Author J.F.Zimmerman
#Updated 5/13/2022
#Code for Particle Imaging Velocimetry (PIV) analysis of ventricle Ejection Fractions
#Takes inputs from the PIV Analysis folder, and phase averages over the samples
#Makes use of the OPENPIV Framework

from openpiv import tools, scaling, pyprocess, validation
import os
import ntpath
import openpiv
import numpy as np
from matplotlib.pyplot import *
from mpl_toolkits.mplot3d.axes3d import Axes3D
%matplotlib inline
import glob
import PIVfilters
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters
import scipy.fftpack
from scipy import optimize
from scipy import stats

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

import pyximport
pyximport.install()

In [ ]:
def sortGlob(s):
    return int(os.path.basename(s)[:-4])

def movAvg(x,N):
    return np.convolve(x, np.ones((N,))/N, mode='valid')

def cos_func(x, a, b,c,FPS=30):
    return a*np.cos(b*2*3.141592*(x+c)/FPS-3.141592/2)

def sin_func(x, a, b,c,FPS=30):
    return a*np.sin(b*2*3.141592*(x+c)/FPS)

    
def sin_func15(x, a, b,c,FPS=15):
    return a*np.sin(b*2*3.141592*(x+c)/FPS)


def sin_func22(x, a, b,c,FPS=22):
    return a*np.sin(b*2*3.141592*(x+c)/FPS)


def sin_funcU(x, a, b,c,FPS):
    return a*np.sin(b*2*3.141592*(x+c)/FPS)



    
class vectorField:
    def __init__(self,U,V,timepoint=0):
        self.U = U
        self.V = V
        self.size = U.size
        self.shape = U.shape
        self.timepoint = 0
        self.width = U.shape[1]
        self.height = U.shape[0]
    
    def loadVectorField(filepath,filetype ='*.txt'):
        vectorArray = np.array([])
        files = glob.glob(filepath+filetype)
        files.sort(key=sortGlob)
        i = 0
        

        print(f'Len Files: {len(files)}')
        for doc in files:
            print (f'Loading: {doc}')
            z = np.loadtxt(doc)
            if i ==0:
                width = len(np.unique(z[:,1]))
                height = int(z[:,1].size/width)
                x = np.reshape(z[:,0],(width,height))
                y = np.reshape(z[:,1],(width,height))
            zu = np.reshape(z[:,2],(width,height))
            zv = np.reshape(z[:,3],(width,height))
            vect = vectorField(zu,zv,timepoint=i)
            vectorArray = np.append(vectorArray,vect)
            i+=1
        return x,y, vectorArray
    
    def loadSingleVectorField(filepath,filetype ='*.txt'):
        vectorArray = np.array([])
        files = glob.glob(filepath+filetype)
        files.sort(key=sortGlob)
        i = 0
        
        for doc in files:
            print (f'Loading: {doc}')
            z = np.loadtxt(doc)
            #print(z.size)
            if i ==0:
                width = len(np.unique(z[:,1]))
                height = int(z[:,1].size/width)
                x = np.reshape(z[:,0],(width,height))
                y = np.reshape(z[:,1],(width,height))
            zu = np.reshape(z[:,2],(width,height))
            zv = np.reshape(z[:,3],(width,height))
            vect = vectorField(zu,zv,timepoint=i)
            vectorArray = np.append(vectorArray,vect)
            i+=1
        return x,y, vectorArray

def getMag(u,v):
    return np.sqrt(u*u+v*v)

def bg_colormap():
    #New Gray-> Blue Colormap
    cbits = 256
    vals = np.ones((cbits, 4))
    vals[:, 0] = np.linspace(245./256,20./256, cbits)
    vals[:, 1] = np.linspace(245./256,20./256, cbits)
    vals[:, 2] = np.linspace(245./256,120./256,  cbits)
    newcmp = ListedColormap(vals)
    return newcmp

def getVMags(varray):
    magarray = np.array([])
    for i in range(0,np.size(varray)):
        #mag = np.sum(varray[i].V-np.median(varray[i].V))
        mag = np.sum(varray[i].V)
        magarray = np.append(magarray,mag)
    return magarray

def getUMags(varray):
    magarray = np.array([])
    for i in range(0,np.size(varray)):
        mag = np.sum(varray[i].U)
        magarray = np.append(magarray,mag)
    return magarray

def getAbsMags(varray):
    magarray = np.array([])
    for i in range(0,np.size(varray)):
        mag = np.sum(np.sqrt(varray[i].U**2+varray[i].V**2))
        magarray = np.append(magarray,mag)
    return magarray

def findPeakFreq(varray,s=5,FPS=30):
    #Finds the stimulation frequnecy in seconds, and the total frames per complete cycles
    vmag = getVMags(varray)
    Transform = np.fft.fft(vmag,norm="ortho")
    freq = np.fft.fftfreq(vmag.shape[-1],d=1/FPS)
    stop = int(round(freq.shape[0]/2))
    peakfreq = freq[np.where(Transform.real[1:stop]**2==np.max(Transform.real[1:stop]**2))[0]+1]
    frames = peakfreq*FPS
    print(freq)
    return peakfreq, int(np.round(frames))

def getPeakShift(avgVarray,peakfreq,rframes,s=3,FPS=30,fit_func=sin_func):
    #Finds how much the cycle is shifted from the downstroke, returns number of frames (-rframe to rframe)
    frames = rframes
    cut = varray.shape[0]%rframes
    
    vmags = np.array([])
    for i in range(0,avgVarray.shape[-1]):
        mag = np.sum(avgVarray[:,:,i])
        vmags = np.append(vmags,mag)
    
    vmags = vmags - np.average(vmags)
    if np.isnan(np.sum(vmags)):
        print(vmags)
    vmag_x = np.linspace(1,vmags.size,vmags.size)
    params, params_covariance = optimize.curve_fit(fit_func, vmag_x, vmags, p0=[vmags.max(), 1/peakfreq[0], 0,FPS], bounds=([vmags.max()*.7, (1/peakfreq[0])*.5, -frames,FPS*0.999999], [vmags.max()*1.5, (1/peakfreq[0])*1.5, frames,FPS*1.00000001]))
    return int(np.round(params[2]))

def avgOverCycles(varray,shift,rframes):
    #Shift frames by a given amount
    nvarray = np.roll(varray[:],(shift%rframes))
    
    #temporary array to hold results
    avgUarray = np.zeros((varray[0].shape[0],varray[0].shape[1],rframes))
    avgVarray = np.zeros((varray[0].shape[0],varray[0].shape[1],rframes))
    for i in range (0,rframes):
        print(f'Processing {i} frame')
        checksum = i%rframes
        cycles = 0
        for k in range(0,nvarray.size):
            if k%rframes == checksum:
                cycles+=1
        Uarray = np.zeros((varray[0].shape[0],varray[0].shape[1],cycles))
        Varray = np.zeros((varray[0].shape[0],varray[0].shape[1],cycles))
        j= 0

        for k in range(0,nvarray.size):
            if k%rframes == checksum:
                Uarray[:,:,j] = nvarray[k].U
                Varray[:,:,j] = nvarray[k].V
                j+=1
        u = np.median(Uarray,2)
        v = np.median(Varray,2)
        avgUarray[:,:,i] =  u
        avgVarray[:,:,i] =  v
    
    return avgUarray,avgVarray

def halfellipseVolume(a,b,c):
    return np.pi*(4./3.)*a*b*c*(1/2)

In [ ]:
#Mass Frame Average of Ventricle Contraction, Assumes No Background Drift Correct

#--------------- Inputs ----------------------
loadpath = 'Data\\Analyzed\\' #Location of input files, should be number .txt, with stored velocity fields produced in the PIV analysis
savepath = 'Data\\Averaged\\' #Location for saving the files

FPS = 15 #30 frames person second, 1/2 if sample is downsized

Hz = 1.0 #Beating frequency of the ventricle (stimulation frequency)

edgf = 2 #Edges to filter out for the flux processing

rho = 978 #Density of the fluid kg/m^3
skip = 5 #Velocity vectors to skip in graphing, will still be processed/saved
vscale = 12.5 #

#Function to fit to get your wavelength shift (alinging systole/diastole)
fit_func = sin_func15 # options include sin_func (for 30 fps),sin_func15 (for 15 fps), sin_func22 (for 22 fps) cos_func 

#Grapphing considerations
pad = 6 #Padding on the side of the analysis
figheight= 5 #Height of the graphed figure
vlowlim,vuplim = 0,2 #Upper and lower bounds on the velocity mapping graphs
s = 3 #Smoothing factor for the underlying mesh

#Unit Conversion
mmconv = 1./1000. #Assumes input in mm and mm/s
#--------------- Functional Code ----------------------

root = glob.glob(loadpath+'*')
print(root[-1])
print (f'Filepath Size {len(root)}')

#Background color for image graphing
cms = bg_colormap()

#Setup plot fonts/ label sizes
font = {'family':'Arial', 'weight':'bold','size':24}
matplotlib.rc('font',**font)
matplotlib.rc('xtick',labelsize=15)
matplotlib.rc('ytick',labelsize=15)

#Searchs for files in each condition and each sample
for angle in root[:]:
    print('Searching in..')
    print(os.path.basename(angle))
    subfolders = glob.glob(angle+'\\*')

    
    if os.path.isdir(savepath+os.path.basename(angle)):
        print('Save Folder Confirmed')
    else:
        print('Making a new Save Folder')
        os.mkdir(savepath+os.path.basename(angle))
    
    for cant in subfolders:
        
        #Making new temp array to store fluid flux
        FluxX = np.array([])
        FluxY = np.array([])
        
        if os.path.isdir(savepath+os.path.basename(angle)+'\\'+os.path.basename(cant)):
            print('SaveCant-Confirm')
        else:
            os.mkdir(savepath+os.path.basename(angle)+'\\'+os.path.basename(cant))
        
        #Loading Sample #
        print (f'Loading: {cant}')
        x1,y1,varray = vectorField.loadVectorField(cant+'\\','*.txt')
        mask = np.ones(varray[0].shape, dtype=bool)

        #Finding the peak frequency of contraction, and the total number of frames per cycle
        peakfreq = np.array([Hz])
        rframes = int(np.round(FPS/Hz))
        
        #Phase Average over the cycle
        avgUarray,avgVarray = avgOverCycles(varray,0,rframes)
        
        #Shift the cycle average so that analysis/graphing starts at systole
        shift = getPeakShift(avgVarray,peakfreq,rframes,s=s,FPS=FPS,fit_func=fit_func)
        avgUarray = np.roll(avgUarray,shift,axis=-1)
        avgVarray = np.roll(avgVarray,shift,axis=-1)
        magarray = np.sqrt(avgUarray**2+avgVarray**2)
        
        for i in range(0,rframes):
            
            #Generate Background Mesh for averaged velocity fields
            XYUV = np.dstack((x1.ravel(),y1.ravel(),avgUarray[:,:,i].ravel(),avgVarray[:,:,i].ravel()))[0,:,:]

            xnodes = np.unique(XYUV[:,0]).shape[0]
            ynodes = np.unique(XYUV[:,1]).shape[0]
            AR = xnodes/float(ynodes)
            
            #Coordinate system for background mesh
            x = np.linspace(np.min(XYUV[:,0]),np.max(XYUV[:,0]),xnodes)
            y = np.linspace(np.min(XYUV[:,1]),np.max(XYUV[:,1]),ynodes)
            dx = x[1]-x[0] ## given in mm
            dy = y[1]-y[0] ## given in mm
            #Setup actual mesh
            xv,yv = np.meshgrid(x,y)

            #Average over velocity data to find values for mesh grid
            u = scipy.interpolate.griddata(XYUV[:,0:2],XYUV[:,2],(xv,yv)) #-BKG_u
            v = scipy.interpolate.griddata(XYUV[:,0:2],XYUV[:,3],(xv,yv)) #-BKG_v

            #Determine the total flux inside the velocity field
            #Padding indicates distance away from the frame's edge, avoid edge inconsistancies
            fluxX = np.sum(u[:,-pad]*dy)+np.sum(u[:,pad]*dy)
            fluxY = np.sum(v[pad,:]*dx)+np.sum(v[-pad,:]*dx)
            
            #Normalize to SI units
            FluxX = np.append(FluxX,rho*fluxX*mmconv*mmconv) #kg/m*s
            FluxY = np.append(FluxY,rho*fluxY*mmconv*mmconv)
            
            #Smooth Grid
            x = np.linspace(np.min(XYUV[:,0]),np.max(XYUV[:,0]),xnodes*s)
            y = np.linspace(np.min(XYUV[:,1]),np.max(XYUV[:,1]),ynodes*s)
            xs,ys = np.meshgrid(x,y)
            
            #Velocity Map Smoothed coordinate grid for mapping
            M = np.sqrt(XYUV[:,2]**2+XYUV[:,3]**2)
            gridinterp = scipy.interpolate.griddata(XYUV[:,0:2],M,(xs,ys))
            us = scipy.interpolate.griddata(XYUV[:,0:2],XYUV[:,2],(xs,ys),method='linear')
            vs = scipy.interpolate.griddata(XYUV[:,0:2],XYUV[:,3],(xs,ys),method='linear')
            
            #Make Savepath for Velocity Maps
            if os.path.isdir(savepath+os.path.basename(angle)+'\\'+os.path.basename(cant)+'\\Velocity') == False:
                os.mkdir(savepath+os.path.basename(angle)+'\\'+os.path.basename(cant)+'\\Velocity')
            
            #Plot Velocity Map
            #Setup Figure
            fig, ax = subplots(figsize=(figheight*AR,figheight))
            normv = matplotlib.colors.Normalize()
            normv.autoscale(np.array([vlowlim,vuplim]))
            vsm = matplotlib.cm.ScalarMappable(cmap=cms,norm=normv)
            vsm.set_array([])
            
            #Plot actual values
            ax.pcolormesh(xs,ys, gridinterp,cmap=cms,vmin=vlowlim,vmax=vuplim)      
            q = ax.quiver(xs[::skip,::skip], ys[::skip,::skip],us[::skip,::skip], vs[::skip,::skip], color='k',clim=(vlowlim,vuplim),width=0.0025,scale=vscale)
            
            #Label coordinate axis/ color bars/ adjust layout
            ax.set_xlabel('X (mm)',size=30,labelpad=20)
            ax.set_ylabel('Y (mm)',size=30,labelpad=20)
            fig.cbar = fig.colorbar(vsm,cmap=cms)
            fig.cbar.set_label(('Velocity (mm/s)'), rotation=270,fontsize=30,labelpad=30)
            tight_layout()

            #Save and Close Velocity map
            savefig(savepath+os.path.basename(angle)+'\\'+os.path.basename(cant)+'\\Velocity\\'+str(i)+'.png',DPI=200)
            clf()
            close()
            
            #Save Results from individual frames
            np.savetxt(savepath+os.path.basename(angle)+'\\'+os.path.basename(cant)+'\\XYUV'+str(i)+'.txt',XYUV)
        
        #Save the overall sample flux
        FluxYraw = FluxY.copy()
        
        #Baseline subtract
        FluxX = FluxX-FluxX[0]
        FluxY = FluxY-FluxY[0]
        
        #Plot Sample Flux
        flux_t = np.linspace(0,1/Hz,FluxY.size)
        fig, ax = subplots(figsize=(figheight*1.2,figheight))
        ax.plot(flux_t,FluxY*1000,'orange',linewidth=3)
        ax.plot([0,1],[0,0],'k--',linewidth=3)
        rcParams['svg.fonttype'] = 'none'
        xlim(0,1)
        xlabel('Time(s)',size=28,weight='bold')
        ylabel(r'N$_{flux}$ (g/m$\cdot$s)',size=28,weight='bold')
        savefig(savepath+os.path.basename(angle)+'\\'+os.path.basename(cant)+'\\FluxY.svg', format='svg')
        #Remove figure, from python to prevent memory overload
        clf()
        close()
        
        print('Systole FluxY: '+str(np.sum(FluxY[0:int(np.round(FPS/2))])))
        
        #Save the overall sample flux values
        np.savetxt(savepath+os.path.basename(angle)+'\\'+os.path.basename(cant)+'\\FluxY.txt',FluxYraw)
        

In [ ]:
#Process Ejection Fractions and Cardiac Output

#--------------- Inputs ----------------------
#Takes inputs from the frame averages given above, and creates analyzed plots

loadpath = 'Data\\Averaged\\'
savepath = 'Data\\'

FPS=15 #Frames per second, if downsample from 30 fps, use 15
hp = 7 #halfway point - frame for the end of systole

rho = 978. #kg/m^3 Fluid density of testing solution

#Values for the Ventricle volume
c = 9.68*2 #semi major axis, units in MM
a,b = 6.02,3.01 # semi-minor axes, units of mm

ellipse_volume = np.pi*(2./3.)*a*b*c
circ_corc = np.pi/4 #Correction for the area of the basal plane occupied by the ventricular opening

#Smoothing function
s= 3

#Boxplots Displays
lw = 3 #Linewidth
color = [0,0,1,0.8] #marker color
ms = 16 #marker size
ts = 20 #titlesize

#--------------- Functional Code ----------------------
#Setup Analysis
root = glob.glob(loadpath+'*')
avgFluxY_syst = np.array([])
stdFluxY_syst = np.array([])
sumFluxY_sys1 = np.array([])
sumFluxY_sys2 = np.array([])
dt = 1.0/FPS

#Setup Fonts
font = {'family':'Arial', 'weight':'bold','size':24}
matplotlib.rc('font',**font)
matplotlib.rc('xtick',labelsize=15)
matplotlib.rc('ytick',labelsize=15)

j = 0
for angle in root[:]:
    print('Searching in..')
    print(os.path.basename(angle))
    subfolders = glob.glob(angle+'\\*')
    #print(subfolders)
    
    sumFluxY_sys = np.array([])
    ef_sys = np.array([])
    sys_vol = np.array([])
    CO_mm3_sys = np.array([])
    for cant in subfolders:
        FluxY = np.loadtxt(loadpath+os.path.basename(angle)+'\\'+os.path.basename(cant)+'\\FluxY.txt')
        t = np.linspace(0,1,FluxY.size)
        
        
        #Baseline subtract over flux, and smooth output functions
        FluxY = np.roll(FluxY,1)
        FluxY=FluxY-FluxY[-1]
        FluxY = FluxY - t*(FluxY[-1]-FluxY[0])
        FluxY = movAvg(FluxY,s)
        t = np.linspace(0,1,FluxY.size)
        
        
        ellipse_volume = halfellipseVolume(a,b,c) #Volume of a half-ellipsoid of revolution (i.e. ventricle shape)

        
        print(cant)
        #Make sure there are no errors in data
        if np.isnan(np.sum(FluxY[:hp])):
            print(true)
        if np.isinf(np.sum(FluxY[:hp])):
            print(true)
        sumFluxY_sys =  np.append(sumFluxY_sys,np.sum(FluxY[:hp]*dt)*1000)
        
        
        #Ejection Fraction Calc
        co_mm3 = sumFluxY_sys[-1]* (a*2./1000.)*circ_corc*(rho/1000.)*1000
        ef = sumFluxY_sys[-1]* (a*2./1000.)*circ_corc*(rho/1000.)*1000./ellipse_volume #Output Flux * Total height *Circ inscirbed sqaure*density *g to mm^3 conversion
        ef = ef*100 #Convert to percentage
        ef_sys = np.append(ef_sys,ef)
        sys_vol = np.append(sys_vol,ellipse_volume)
        CO_mm3_sys = np.append(CO_mm3_sys,co_mm3)

    sumFluxY_sys = np.sort(sumFluxY_sys)
    ef_sys = np.sort(ef_sys)
    CO_mm3_sys = np.sort(CO_mm3_sys)
    
    #If additional conditions are measured, will need to adjust this code
    #to work for more examples, but creating an additional j conditoin, and additional storage arrays.
    #-------------------Setup for 2 conditions -----------
    
    #If condition 1, store in the first condition
    if j == 0:
        sumFluxY_sys1 = sumFluxY_sys.copy()
        ef1 = ef_sys.copy()
        co_mm3_1  =co_mm3.copy()
    
    #If a second condition, store there
    else:
        sumFluxY_sys2 = sumFluxY_sys.copy()
        ef2 = ef_sys.copy()
        co_mm3_2  =co_mm3.copy()
        
    #------------ End Setup for 2 conditions -------------
    
    print("SumFluxY:")
    print(sumFluxY_sys)
    print("EF system:")
    print(ef_sys)
    print("CO_mm3:")
    print(CO_mm3_sys)
    print(f'Avg FluxY: {np.average(sumFluxY_sys)}')
    print(f'Std FluxY: {np.std(sumFluxY_sys)}')
    print(f'Avg COmm3: {np.average(CO_mm3_sys)}')
    print(f'STD COmm3: {np.std(CO_mm3_sys)}')
    print(f'Avg ef: {np.average(ef_sys)}')
    print(f'Std ef: {np.std(ef_sys)}')

    np.savetxt(savepath + "Avg_FluxY"+str(j)+".txt",np.array([np.average(sumFluxY_sys)]))
    np.savetxt(savepath + "StD_FluxY"+str(j)+".txt",np.array([np.std(sumFluxY_sys)]))
    np.savetxt(savepath + "Avg_CO"+str(j)+".txt",np.array([np.average(CO_mm3_sys)]))
    np.savetxt(savepath + "STD_CO"+str(j)+".txt",np.array([np.std(CO_mm3_sys)]))
    np.savetxt(savepath + "Avg_EF"+str(j)+".txt",np.array([np.average(CO_mm3_sys)]))
    np.savetxt(savepath + "STD_EF"+str(j)+".txt",np.array([np.std(CO_mm3_sys)]))
    j+=1
    


#Boxplot Results
boxprops = dict(linewidth=lw)
np.random.seed(1234)

fig = figure(figsize=(12,6))
ax = subplot(1,2,1)
bp = ax.boxplot([sumFluxY_sys1,sumFluxY_sys2],boxprops=boxprops)
ax.set_xticklabels(['Condition1','Condition2'],fontsize=ts*0.75,weight='bold')
ax.set_ylabel(r'Cardiac Output (N$_{flux}$, g/m)',fontsize=ts,weight='bold')
ax.set_xlabel('Sample Conditions',fontsize=ts,weight='bold')

for whisker in bp['whiskers']:
    whisker.set(linewidth=lw)
    
for cap in bp['caps']:
    cap.set(linewidth=lw)
    
for median in bp['medians']:
    median.set(linewidth=lw)

ax2 = subplot(1,2,2)
bp = ax2.boxplot([ef1,ef2],boxprops=boxprops)
ax2.set_xticklabels(['Condition1','Condition2'],fontsize=ts*0.75,weight='bold')
ax2.set_ylabel(r'Ejection Fraction (%)',fontsize=ts,weight='bold')
ax2.set_xlabel('Sample Conditions',fontsize=ts,weight='bold')

for whisker in bp['whiskers']:
    whisker.set(linewidth=lw)
    
for cap in bp['caps']:
    cap.set(linewidth=lw)
    
for median in bp['medians']:
    median.set(linewidth=lw)

#Superimpose scatter over boxplots
y = sumFluxY_sys1
x = np.random.normal(1,0.025,size=len(y))
ax.plot(x,y,'.',ms=ms, c=color)

y = sumFluxY_sys2
x = np.random.normal(2,0.025,size=len(y))
ax.plot(x,y,'.',ms=ms,c = color)

y = ef1
x = np.random.normal(1,0.025,size=len(y))
ax2.plot(x,y,'.',ms=ms, c=color)

y = ef2
x = np.random.normal(2,0.025,size=len(y))
ax2.plot(x,y,'.',ms=ms,c = color)

#Adjust plot Layouts
subplots_adjust(top=0.92, bottom=0.08, left=0.1, right=0.95,hspace=0.25,wspace=0.27)

savefig(savepath+"ConditionsBoxPlots.svg", format="svg")

#Statistical Analysis, t-test
print(f'T-Value:{stats.ttest_ind(sumFluxY_sys1,sumFluxY_sys2)}')
print(f'T-Value:{stats.ttest_ind(ef1,ef2)}')

np.savetxt(savepath + "EF_TTest"+str(j)+".txt",stats.ttest_ind(sumFluxY_sys1,sumFluxY_sys2))
np.savetxt(savepath + "Flux_TTest"+str(j)+".txt",stats.ttest_ind(ef1,ef2))